# The Real code

In [1]:
using Random, Distributions, Plots, XLSX

What needs to be done:
. Statistics along the way
. the simulation gives a shit ton of unfullfilled values, seems fucked!
. Last simulation
... Plots


Questions
. Weird arrival at midnight
. Why the large amount of unfullfilled values?
. Should we keep values for every step at every run or just the final statistics at every run?


In [2]:
### Initial functions to load lambda and mu from the ArrivalProfiles.xlsx

# Change the type to Float64 from Int and String for the loaded vectors
function type_change(v)
    v_ny = Vector{Float64}()
    for i in 1:length(v)
        if typeof(v[i]) == String
            push!(v_ny,parse(Float64,v[i]))
        elseif typeof(v[i]) == Int64
            push!(v_ny,float(v[i]))
        else
            push!(v_ny,v[i])
        end
    end
    return v_ny
end


function load_arri(path="C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    xf = XLSX.readxlsx(path) # Loading

    # Lambda is given as expected arrivals therefore inverse
    lambda_port = 1 ./ type_change(xf[1]["C2:AA2"])
    lambda_dtu = 1 ./ type_change(xf[1]["C3:AA3"])

    # Travel time
    mu_rammer_dtu = type_change(xf[1]["C7:AA7"])
    mu_rammer_port = type_change(xf[1]["C8:AA8"])

    return lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port
end


# Might not be just as fast as PriorityQueue but it is not that much behind
# Inserts a value at the spot it fits
function insert_and_sort!(vect::Vector, x)
    (splice!(vect, searchsorted(vect,x), [x]); vect) 
end


insert_and_sort! (generic function with 1 method)

In [3]:
struct Environment # The environment including arrival rates and travel rates
    lambda_dtu::Vector{Float64}  #arrival rate 1
    mu_rammer_dtu::Vector{Float64} #service rate 1
    lambda_port::Vector{Float64} # arrival rate 2
    mu_rammer_port::Vector{Float64} #service rate 2
    
    # loading the vectors from excel
    lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port = load_arri() # NEEDS the path to load the document from!
    Environment() = new(lambda_port, lambda_dtu, mu_rammer_dtu, mu_rammer_port)
end

mutable struct Stats # Vector including [Station 1, Station 2]
    n_cars::Vector{Int64} # Number of cars at the stations at any given time
    un_fullfilled::Vector{Int64} # Amount of times a customer arrives to an empty station
    fullfilled::Vector{Int64} # Amount of times a costumer arrives and gets a car
    empty_time::Vector{Float64} # The time any given station has 0 cars
    currently_empty::Vector{Bool} # Is the station empty
end

struct Balance
    desired_n_st1::Int64
    desired_n_st2::Int64
    rebalance_times::Vector{Float64}
end


In [21]:

function time_change(e::Array{Tuple{Float64,String},1},k::Int64,day::Int64) # problems
    # Switch day and reset index
    if k == 25
        k = 0
        day += 25
    end
    # Insert the next time change
    popfirst!(e)[1]
    insert_and_sort!(e,(k+day,"time_change"))

    # Change index for time
    k += 1
    return k,day
end

function arrival_student(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats,k::Int64)
    
    # Insert next arrival based on previous arrival
    if station == 1
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st1"))
    elseif station == 2
        insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k]))+e[1][1],"st2"))
    end

    if stat.currently_empty[station] == true # Student arrives at empty station. That is one unsatisfied costumer :(
        stat.un_fullfilled[station] += 1
        popfirst!(e)
        
    else # Costumer leaves station
        stat.n_cars[station] -= 1 # removing car from station
        stat.fullfilled[station] += 1 # one fullfilled request
        
        # Insert travel time based on arrival station
        if station == 1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2")) 
        elseif station == 2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st1")) 
        end

        if stat.n_cars[station] == 0 # station becomes empty
            stat.empty_time[station] -= popfirst!(e)[1] # Empty station timer starts 
            stat.currently_empty[station] = true
        else
            popfirst!(e)
        end
    end 

end 

function arrival_vehicle(station::Int64,e::Array{Tuple{Float64,String},1},env::Environment,stat::Stats)

    if stat.currently_empty[station] == true # Arriving at an empty station
        stat.n_cars[station] += 1 # arrive
        stat.empty_time[station] += popfirst!(e)[1] # End empty timer
        stat.currently_empty[station] = false
    else
        stat.n_cars[station] += 1 # arrive
        popfirst!(e)
    end

end


function rebalancing(rebal::Balance,stat::Stats,env::Environment,e::Array{Tuple{Float64,String},1},k::Int64)
    # NEEDS Documentation and a serious debugging
    if stat.n_cars[2] >= rebal.desired_n_st1
        for i in 1:rebal.desired_n_st1
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1")) 
            stat.n_cars[2] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[2] == 0
            stat.empty_time[2] -= e[1][1]
            stat.currently_empty[2] = true
        end

    else
        for i in 1:stat.n_cars[2]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_dtu[k]))+e[1][1],"que_st1"))
            stat.n_cars[2] -= 1
        end
        if stat.currently_empty[2] == false
            stat.empty_time[2] -= e[1][1]
            stat.currently_empty[2] = true
        end
    end

    # For station 1
    if stat.n_cars[1] >= rebal.desired_n_st2
        for i in 1:rebal.desired_n_st2
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2"))
            stat.n_cars[1] -= 1
        end
        
        # Start empty timer
        if stat.n_cars[1] == 0
            stat.empty_time[1] -= e[1][1]
            stat.currently_empty[1] = true
        end
    else
        for i in 1:stat.n_cars[1]
            insert_and_sort!(e,(rand(Exponential(env.mu_rammer_port[k]))+e[1][1],"que_st2"))
            stat.n_cars[1] -= 1
        end
        if stat.currently_empty[1] == false
            stat.empty_time[1] -= e[1][1]
            stat.currently_empty[1] = true
        end
    end 

    insert_and_sort!(e,(24.0+popfirst!(e)[1],"rebalance"))
    
end



rebalancing (generic function with 1 method)

In [39]:
function sim(days_to_run::Int64,initial_n_cars::Vector{Int64}, rebal::Balance, env::Environment)

    stat = Stats([initial_n_cars[1],initial_n_cars[2]],[0,0],[0,0],[0,0],[false,false])

    e = Array{Tuple{Float64,String},1}()
    
    runs = 0
    k = 1
    day = 0
    insert_and_sort!(e,(rand(Exponential(env.lambda_dtu[k])),"st1")) # Insert first arrival
    insert_and_sort!(e,(rand(Exponential(env.lambda_port[k])),"st2")) # Insert frist arrival
    insert_and_sort!(e,(0.0,"time_change")) # THIS NEEDS TO BE FIXED!!!!!!!!

    for i in 1:length(rebal.rebalance_times)
        insert_and_sort!(e,(rebal.rebalance_times[i],"rebalance"))
    end


    while e[1][1] <= days_to_run*24.0 && runs <= 100000
        load_type = e[1][2]
        if load_type == "st1"
            arrival_student(1,e,env,stat,k)
        elseif load_type == "st2"
            arrival_student(2,e,env,stat,k)
        elseif load_type == "que_st1"
            arrival_vehicle(1,e,env,stat)
        elseif load_type == "que_st2"
            arrival_vehicle(2,e,env,stat)
        elseif load_type == "time_change"
            k,day = time_change(e,k,day)
        elseif load_type == "rebalance"
            rebalancing(rebal,stat,env,e,k)
        else
            throw(ErrorException("Issue in the event type!"))
        end
        runs += 1 # Checks the amount of events
    end
    
    # Make sure the empty timer stops at the end of the run
    print(stat.currently_empty,stat.n_cars)
    if stat.currently_empty[1] == true
        stat.empty_time[1] += e[1][1]
    elseif stat.currently_empty[2] == true
        stat.empty_time[2] += e[1][1]
    end
    
    return stat.n_cars,stat.un_fullfilled,stat.fullfilled,stat.empty_time,runs
end

sim (generic function with 1 method)

### Testing

In [40]:

function main(initial_n_cars::Vector{Int64},days_to_run::Int64,n_of_runs::Int64,rebalance::Tuple{Int64, Int64, Vector{Float64}},path = "C:/Users/rasmu/Desktop/OneDrive/OneDrive - Danmarks Tekniske Universitet/Decisions under uncertainty/Projekt/ArrivalProfiles.xlsx")
    # Rebalance is given in [desired_n_st1, desired_n_st2, time_1, time_2] for now int
    env = Environment()
    rebal = Balance(rebalance[1],rebalance[2],rebalance[3])
    overview = []
    efficiancy = []

    for i in 1:n_of_runs
        a = sim(days_to_run,initial_n_cars,rebal,env)
        overview = [overview;a]
        # NEeds final stats
    
        efficiancy_fullfilled = [a[3][1]/(a[3][1]+a[2][1]), a[3][2]/(a[3][2]+a[2][2]) ]
        efficiancy_emptytime = [a[4][1]/(days_to_run*24),a[4][2]/(days_to_run*24)]

        print("Efficiancy percentage for station 1 is ")
        printstyled(round(efficiancy_fullfilled[1]*100,digits=2),"%"; color=:blue)
        print(" and ")
        printstyled(round(efficiancy_fullfilled[2]*100,digits=2),"%"; color=:blue)
        print(" for station 2. Percentage of time spent empty for station 1 is ")
        printstyled(round(efficiancy_emptytime[1]*100,digits=2),"%"; color=:blue)
        print(" and ")
        printstyled(round(efficiancy_emptytime[2]*100,digits=2),"%"; color=:blue)
        println(" for station 2.")

        efficiancy = [efficiancy; [efficiancy_fullfilled]]
    end
    
    return overview, efficiancy
end

main (generic function with 2 methods)

In [56]:
main([80,20],100,20,(80,20,[4.0]))

Bool[0, 0][80, 19]Efficiancy percentage for station 1 is 76.91% and 49.73% for station 2. Percentage of time spent empty for station 1 is 6.92% and 17.66% for station 2.
Bool[0, 1][90, 0]Efficiancy percentage for station 1 is 73.24% and 51.52% for station 2. Percentage of time spent empty for station 1 is 7.66% and 16.6% for station 2.
Bool[0, 0][36, 58]Efficiancy percentage for station 1 is 79.57% and 70.39% for station 2. Percentage of time spent empty for station 1 is 5.88% and 9.84% for station 2.
Bool[0, 1][100, 0]Efficiancy percentage for station 1 is 76.37% and 67.13% for station 2. Percentage of time spent empty for station 1 is 7.3% and 13.25% for station 2.
Bool[0, 0][90, 1]Efficiancy percentage for station 1 is 76.2% and 62.4% for station 2. Percentage of time spent empty for station 1 is 7.26% and 13.58% for station 2.
Bool[0, 0][15, 77]Efficiancy percentage for station 1 is 79.43% and 58.43% for station 2. Percentage of time spent empty for station 1 is 6.21% and 17.04% fo

(Any[([80, 19], [2248, 5256], [7487, 5199], [166.13252981504093, 423.86921977171437], 40749), ([90, 0], [2471, 4839], [6763, 5142], [183.74569971770188, 398.33041178581357], 38634), ([36, 58], [2313, 2769], [9006, 6582], [141.00231666216496, 236.14288328764178], 44383), ([100, 0], [2686, 3027], [8681, 6181], [175.31870466036344, 318.0386383981372], 43626), ([90, 1], [2460, 3492], [7875, 5795], [174.34741685906647, 325.8138712389882], 41247), ([15, 77], [2027, 4491], [7825, 6313], [149.1159823339658, 408.84363430491], 42098), ([67, 28], [2313, 3259], [8696, 6266], [172.11754602823385, 243.98527428628358], 43588), ([83, 0], [3196, 4076], [8932, 6195], [250.71755574637382, 342.14258534425653], 45609), ([81, 9], [2726, 3953], [8051, 5818], [212.66999106715048, 331.3012687830278], 42225), ([100, 0], [2328, 4912], [7554, 5430], [153.6272092315321, 399.890315868008], 41025), ([47, 45], [4205, 2900], [9192, 5745], [292.2651518204748, 229.8676603613835], 45538), ([90, 0], [2374, 3762], [7161, 5

In [45]:
average_effic = []
for n in 10:10
    for i in 1:(n-1)
        _, effic = main([i,n-i],100,n,(n,n,[2.1]))
        total_effic = [effic[1][1],effic[1][2]]
        for j in 2:n
            total_effic = [effic[j][1]+total_effic[1],effic[j][2]+total_effic[2]]
        end
        
        average_effic = [average_effic; (total_effic[1]/n,total_effic[2]/n,[i,n-i,n])]

    end
end

display(average_effic)


Bool[0, 1][10, 0]Efficiancy percentage for station 1 is 30.48% and 35.48% for station 2. Percentage of time spent empty for station 1 is 31.33% and 23.6% for station 2.
Bool[0, 0][2, 1]Efficiancy percentage for station 1 is 29.23% and 27.74% for station 2. Percentage of time spent empty for station 1 is 28.88% and 24.88% for station 2.
Bool[1, 0][0, 8]Efficiancy percentage for station 1 is 29.68% and 28.62% for station 2. Percentage of time spent empty for station 1 is 21.58% and 28.89% for station 2.
Bool[0, 1][10, 0]Efficiancy percentage for station 1 is 29.8% and 25.98% for station 2. Percentage of time spent empty for station 1 is 25.86% and 30.28% for station 2.
Bool[0, 0][1, 9]Efficiancy percentage for station 1 is 28.66% and 32.13% for station 2. Percentage of time spent empty for station 1 is 27.18% and 26.03% for station 2.
Bool[1, 0][0, 9]Efficiancy percentage for station 1 is 27.51% and 32.82% for station 2. Percentage of time spent empty for station 1 is 30.26% and 24.31% f

9-element Vector{Any}:
 (0.29410689035858173, 0.297099973714128, [1, 9, 10])
 (0.28989483046097914, 0.29719974905894825, [2, 8, 10])
 (0.30059879726653416, 0.3009807016645129, [3, 7, 10])
 (0.3101118515242055, 0.2833625086971167, [4, 6, 10])
 (0.30092347938118624, 0.2967755965637794, [5, 5, 10])
 (0.2987711238933245, 0.3095876122895072, [6, 4, 10])
 (0.3117719774677475, 0.29921571453683066, [7, 3, 10])
 (0.2963123471514238, 0.305804739997873, [8, 2, 10])
 (0.303517658683894, 0.31548533716116695, [9, 1, 10])

0][0, 9]Efficiancy percentage for station 1 is 28.78% and 30.16% for station 2. Percentage of time spent empty for station 1 is 29.6% and 24.11% for station 2.
Bool[0, 1][10, 0]Efficiancy percentage for station 1 is 28.04% and 29.03% for station 2. Percentage of time spent empty for station 1 is 26.18% and 27.31% for station 2.
Bool[0, 1][10, 0]Efficiancy percentage for station 1 is 30.19% and 27.63% for station 2. Percentage of time spent empty for station 1 is 24.66% and 31.06% for station 2.
Bool[0, 0][4, 3]Efficiancy percentage for station 1 is 30.57% and 30.31% for station 2. Percentage of time spent empty for station 1 is 24.97% and 25.86% for station 2.
Bool[1, 0][0, 8]Efficiancy percentage for station 1 is 34.06% and 28.74% for station 2. Percentage of time spent empty for station 1 is 23.35% and 26.64% for station 2.
Bool[0, 0][2, 7]Efficiancy percentage for station 1 is 31.51% and 32.58% for station 2. Percentage of time spent empty for station 1 is 23.5% and 24.0% for statio

In [26]:
_, effic = main([50,50],100,20,(50,50,[24.0]))
display(effic)


20-element Vector{Any}:
 [0.4484238567411573, 0.49320023148148145]
 [0.4030700013232764, 0.35443037974683544]
 [0.4108507705801278, 0.3815837937384899]
 [0.440361372237822, 0.31684196857550967]
 [0.5653730488328799, 0.40492137449039023]
 [0.3447701958776085, 0.42779258541313303]
 [0.515014803327224, 0.43676003465203583]
 [0.5889882788402221, 0.527561661976494]
 [0.5954261270154656, 0.48217206411514557]
 [0.4267535692116698, 0.543594177835423]
 [0.4672154017857143, 0.4640101024273888]
 [0.5275334866729806, 0.3698039760878632]
 [0.4029126213592233, 0.47454951992634486]
 [0.3860471005317802, 0.41908914728682173]
 [0.5269509251810137, 0.5926792578973759]
 [0.5196155871511322, 0.3875319511502414]
 [0.5180063820088132, 0.3829623944742901]
 [0.4198291555804509, 0.4586476868327402]
 [0.438944918541505, 0.5181695827725438]
 [0.3830881302671047, 0.5162262084075038]